In [1]:
import pandas as pd
data_path = "../../data/"
trees_us = pd.read_csv(data_path + "greenspace_data/5_million_trees_us_cities.csv")
city_info = pd.read_csv(data_path + "census_code_mappers/us_cities_zips.csv")

/var/folders/5t/fxl9hmt15b9gy4kpwg48tz2w0000gn/T/ipykernel_70962/3331623050.py:3: DtypeWarning: Columns (0,1,7,8,9,10,12,14,16,18,19,20,21,22,23,24,26) have mixed types. Specify dtype option on import or set low_memory=False.
  trees_us = pd.read_csv(data_path + "greenspace_data/5_million_trees_us_cities.csv")


In [2]:
print(trees_us.columns, city_info.columns)

Index(['planted_date', 'common_name', 'scientific_name', 'city', 'state',
       'longitude_coordinate', 'latitude_coordinate', 'location_type',
       'zipcode', 'address', 'location_name', 'diameter_breast_height_CM',
       'condition', 'native', 'diameter_breast_height_binned_CM',
       'greater_metro', 'city_ID', 'tree_ID', 'most_recent_observation',
       'retired_date', 'most_recent_observation_type', 'neighborhood', 'ward',
       'district', 'overhead_utility', 'height_M', 'height_binned_M',
       'percent_population'],
      dtype='object') Index(['zip_code', 'latitude_centroid', 'longitude_centroid', 'city',
       'state_abbr', 'state_name', 'zcta', 'parent_zcta', 'population',
       'density', 'county_fips', 'county_name', 'timezone'],
      dtype='object')


In [3]:
print(len(trees_us), len(city_info))

5660237 33788


In [4]:
#count number of nan values in each column
trees_us.isnull().sum()

planted_date                        5213598
common_name                          892476
scientific_name                      529297
city                                    505
state                                    31
longitude_coordinate                 751709
latitude_coordinate                  751578
location_type                       3469760
zipcode                             4768676
address                             1254389
location_name                       5310350
diameter_breast_height_CM           2785979
condition                           3038500
native                                    0
diameter_breast_height_binned_CM    1574755
greater_metro                             0
city_ID                             2017770
tree_ID                             5660237
most_recent_observation             2127842
retired_date                        5658047
most_recent_observation_type        3198777
neighborhood                        5132935
ward                            

In [5]:
del trees_us["tree_ID"] # all nan
del trees_us["percent_population"] # all nan
del trees_us["ward"] # seems useless, too many nans
del trees_us["district"] # seems useless, too many nans
del trees_us["city_ID"] # seems useless
del trees_us['address'] # seems useless
del trees_us['location_name'] # seems useless
del trees_us['neighborhood'] # seems useless
del trees_us['overhead_utility'] # seems useless
del trees_us['retired_date'] # too few values
del trees_us['most_recent_observation'] # seems useless
del trees_us['most_recent_observation_type'] # seems useless

In [6]:
#capture year from planted_date
pattern = r'(\d{4})'
trees_us['planted_year'] = trees_us['planted_date'].str.extract(pattern, expand=True)

#capture month from planted_date
pattern = r'(^\d{2}/)'
trees_us['planted_month'] = trees_us['planted_date'].str.extract(pattern, expand=True).replace(to_replace = r'/', value = '', regex = True)

#capture day from planted_date
pattern = r'(/\d{2}/)'
trees_us['planted_day'] = trees_us['planted_date'].str.extract(pattern, expand=True).replace(to_replace = r'/', value = '', regex = True)

del trees_us['planted_date']

In [7]:
# remove rows with nan values for city and state, too few, around 500
trees_us = trees_us.dropna(subset=['city', 'state'])

In [8]:
trees_us.longitude_coordinate = trees_us.longitude_coordinate.astype(float)
trees_us.latitude_coordinate = trees_us.latitude_coordinate.astype(float)

# fill mean values for longitude and latitude by city and state
trees_us.longitude_coordinate = trees_us.groupby(['city', 'state'])['longitude_coordinate'].transform(lambda x: x.fillna(x.mean()))
trees_us.latitude_coordinate = trees_us.groupby(['city', 'state'])['latitude_coordinate'].transform(lambda x: x.fillna(x.mean()))


city_crd = city_info[['city', 'latitude_centroid', 'longitude_centroid']]

# make dictionary of city and state to latitude and longitude
city_longitude = dict(zip(city_crd.city, city_crd.longitude_centroid))
city_latitude = dict(zip(city_crd.city, city_crd.latitude_centroid))

trees_us.fillna(value={'longitude_coordinate': trees_us.city.map(city_longitude), 'latitude_coordinate': trees_us.city.map(city_latitude)}, inplace=True)

In [9]:
trees_us.location_type.unique()
trees_us.location_type.fillna(value='no_info', inplace=True)
trees_us.location_type.replace(to_replace='nan', value='no_info', inplace=True)
trees_us.location_type.replace(to_replace='<null>', value='no_info', inplace=True)
trees_us.location_type = trees_us.location_type.astype(str)
trees_us.location_type.unique()

array(['green_space', 'no_info', 'built_environment'], dtype=object)

In [10]:
city_zip = city_info[['city', 'zip_code']]
city_zip = dict(zip(city_zip.city, city_zip.zip_code))

trees_us.zipcode.fillna(value=trees_us.city.map(city_zip), inplace=True)

# fill mode by city and state
trees_us.zipcode = trees_us.groupby(['city'])['zipcode'].transform(lambda x: x.fillna(x.mode()))

mask1 = trees_us.city == 'Washington DC'
mask2 = trees_us.city == 'St. Louis'
trees_us.loc[mask1, 'zipcode'] = 20001
trees_us.loc[mask2, 'zipcode'] = 63101

print(trees_us.zipcode.isnull().sum())
trees_us.dropna(subset=['zipcode'], inplace=True)
mask3 = trees_us.zipcode == 'None'
trees_us.drop(trees_us[mask3].index, inplace=True)
trees_us.zipcode = trees_us.zipcode.astype(int)

7416


In [15]:
print(len(trees_us['common_name'].unique()), len(trees_us['scientific_name'].unique()))

names_1 = trees_us[['common_name', 'scientific_name']].drop_duplicates()
names_2 = trees_us[['scientific_name', 'common_name']].drop_duplicates()
#convert to dictionary
common_scientific = names_1.set_index('common_name').to_dict()['scientific_name']
scientific_common = names_2.set_index('scientific_name').to_dict()['common_name']

#fill na values
# trees_us.scientific_name.fillna(trees_us.common_name.map(common_scientific), inplace=True)
trees_us.common_name.fillna(trees_us.scientific_name.map(scientific_common), inplace=True)

trees_us.common_name = trees_us.common_name.str.lower().astype(str)
# trees_us.common_name.fillna("UNK", inplace=True)

trees_us.scientific_name = trees_us.scientific_name.str.lower().astype(str)
# trees_us.scientific_name.fillna("UNK", inplace=True)

4452 1957


In [11]:
# fill by binned mean
trees_us.diameter_breast_height_CM = trees_us.groupby(['diameter_breast_height_binned_CM'])['diameter_breast_height_CM'].transform(lambda x: x.fillna(x.mean()))

print(trees_us.diameter_breast_height_CM.isnull().sum())

# fill by common_name mean
trees_us.diameter_breast_height_CM = trees_us.groupby(['common_name'])['diameter_breast_height_CM'].transform(lambda x: x.fillna(x.mean()))

print(trees_us.diameter_breast_height_CM.isnull().sum())

# fill by overall mean
trees_us.diameter_breast_height_CM.fillna(value=trees_us.diameter_breast_height_CM.mean(), inplace=True)

print(trees_us.diameter_breast_height_CM.isnull().sum())

1571519
962326
0


In [12]:
# do the same for height
trees_us.height_M = trees_us.groupby(['height_binned_M'])['height_M'].transform(lambda x: x.fillna(x.mean()))

print(trees_us.height_M.isnull().sum())

# fill by common_name mean
trees_us.height_M = trees_us.groupby(['common_name'])['height_M'].transform(lambda x: x.fillna(x.mean()))

print(trees_us.height_M.isnull().sum())

# fill by overall mean
trees_us.height_M.fillna(value=trees_us.height_M.mean(), inplace=True)

print(trees_us.height_M.isnull().sum())

4989438
1739083
0


In [13]:
del trees_us['diameter_breast_height_binned_CM'] # delete the binned columns
del trees_us['height_binned_M'] # delete the binned columns

In [14]:
trees_us.condition.fillna(value='unknown', inplace=True)

In [16]:
trees_us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5650836 entries, 0 to 5660234
Data columns (total 16 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   common_name                object 
 1   scientific_name            object 
 2   city                       object 
 3   state                      object 
 4   longitude_coordinate       float64
 5   latitude_coordinate        float64
 6   location_type              object 
 7   zipcode                    int64  
 8   diameter_breast_height_CM  float64
 9   condition                  object 
 10  native                     object 
 11  greater_metro              object 
 12  height_M                   float64
 13  planted_year               object 
 14  planted_month              object 
 15  planted_day                object 
dtypes: float64(4), int64(1), object(11)
memory usage: 732.9+ MB


In [17]:
trees_us.isnull().sum()

common_name                        0
scientific_name                    0
city                               0
state                              0
longitude_coordinate               0
latitude_coordinate                0
location_type                      0
zipcode                            0
diameter_breast_height_CM          0
condition                          0
native                             0
greater_metro                      0
height_M                           0
planted_year                 5215443
planted_month                5238997
planted_day                  5239431
dtype: int64

In [21]:
df = trees_us.isnull().sum()
stringcols = [col for col in trees_us.columns if trees_us[col].dtype == 'object' and df[col] == 0]
print(stringcols)
# convert to string
trees_us[stringcols] = trees_us[stringcols].astype(str)

['common_name', 'scientific_name', 'city', 'state', 'location_type', 'condition', 'native', 'greater_metro']


In [23]:
trees_us.to_csv(data_path + "greenspace_data/5_million_trees_us_cities_cleaned.csv", index=False)